  - Realice una clasificación de los mismos y evalúe su desempeño utilizando la métrica de evaluación AMS.

In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
import sys
sys.path.append("../../utils")

from HiggsBosonUtils import AMS

In [11]:
INPUT_MODELS_PATH = "./trained-models"
INPUT_DATA_PATH = "../../data/output"
data_training = pd.read_csv('../../data/input/training.csv')
X_train = np.loadtxt(f"{INPUT_DATA_PATH}/X_train.txt")
y_train = np.loadtxt(f"{INPUT_DATA_PATH}/y_train.txt")

weights = np.array(data_training['Weight'])

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

C:\Users\maria\miniconda3\envs\TAA\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [5]:
rf_clf = pickle.load(open(f"{INPUT_MODELS_PATH}/rf.sav", 'rb'))
dt_clf = pickle.load(open(f"{INPUT_MODELS_PATH}/dt.sav", 'rb'))

In [16]:
y_pred_dt = dt_clf.predict(X_train)

s = data_training.loc[(y_pred_dt == 1) & (y_train == 1)].Weight.sum()
b = data_training.loc[(y_pred_dt == 1) & (y_train == 0)].Weight.sum()

print('AMS train (Desicion Tree) = ', AMS(s, b))

AMS train (Desicion Tree) =  67.71112289505676


In [17]:
y_pred_rf = rf_clf.predict(X_train)

s = data_training.loc[(y_pred_rf == 1) & (y_train == 1)].Weight.sum()
b = data_training.loc[(y_pred_rf == 1) & (y_train == 0)].Weight.sum()

print('AMS train (Random Forest) = ', AMS(s, b))

AMS train (Random Forest) =  67.70627582003308


In [18]:
s = data_training.loc[ (y_train == 1)].Weight.sum()
b = 0

print('AMS train (Caso Ideal) = ', AMS(s, b))

AMS train (Caso Ideal) =  67.71112289505676


In [15]:
s = data_training.loc[y_train == 1].Weight.sum()
b = data_training.loc[y_train == 0].Weight.sum()

print('AMS train (todos se predicen como Signal) = ', AMS(s, b))

AMS train (todos se predicen como Signal) =  1.0790735173438217


In [14]:
s = 0
b = 0

print('AMS train (todos se predicen como Background) = ', AMS(s, b))

AMS train (todos se predicen como Background) =  0.0


Peor caso AMS = 0, pq s=0 (train)

 ## Parte 3 -  Un primer pipeline

 - Probar clasificar los datos del proyecto con los distintos métodos vistos en este taller (Árboles de Decisión, Random Forest, XGBoost).

In [9]:
dt_clf = pickle.load(open(f"{INPUT_MODELS_PATH}/dt.sav", 'rb'))

In [10]:
y_pred_dt = dt_clf.predict(X_train)

s = data_training.loc[(y_pred_dt == 1) & (y_train == 1)].Weight.sum()
b = data_training.loc[(y_pred_dt == 1) & (y_train == 0)].Weight.sum()

print('AMS train (desicion tree)= ', AMS(s, b))

AMS train (desicion tree)=  67.71112289505676


In [11]:
xgb_clf = pickle.load(open(f"{INPUT_MODELS_PATH}/xgb.sav", 'rb'))

In [12]:
y_pred_xgb = xgb_clf.predict(X_train)

s = data_training.loc[(y_pred_xgb == 1) & (y_train == 1)].Weight.sum()
b = data_training.loc[(y_pred_xgb == 1) & (y_train == 0)].Weight.sum()

print('AMS train (XGBoost) = ', AMS(s, b))

AMS train (XGBoost) =  3.0588212616821546


 ## Parte 4 -  Un primer pipeline
Descargue la implementación de la métrica de la página del [curso](https://eva.fing.edu.uy/mod/resource/view.php?id=135850). Modifique el nombre del archivo descargado a *HiggsBosonUtils.py* y guarde dicho archivo en una carpeta *tools* en el mismo directorio donde está el presente Notebook. En estas condiciones, puede importar la función *AMS* contenida dentro de *HiggsBosonUtils.py* de la siguiente forma:  

- será necesario también fijar un punto de operación que maximice el desempeño de la métrica, en este caso, el *AMS*. En función de ello, resulta pertinente observar que valores resultan razonables para dicha métrica.

#### Objetivos:

 - Levantar el conjunto de datos del proyecto. 
 - Obtener el desempeño de *AMS* obtenido si el modelo clasifica todos los eventos como *background*.
 - Obtener el desempeño de *AMS* obtenido si el modelo clasifica todos los eventos como *signal*.
 - Obtener el desempeño de *AMS* obtenido si el modelo clasifica perfectamente todos los eventos.
 - Observe los Leaderboard de Kaggle y en base a los resultados obtenidos anteriormente que discuta que valores le resultan razonables de *AMS*.
 - Con los datos del proyecto genere dos conjuntos uno de Entrenamiento y otro de Validación. Luego, entrene el modelo y evalúe el desempeño en el conjunto de Validación. Observe como varía el AMS al modificar el umbral.

In [14]:
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import StratifiedKFold
from HiggsBosonUtils import AMS

In [27]:
def map_s_and_b_values(data):
    data['Label'] = data['Label'].map({'b': 0, 's': 1})
    return data

In [28]:
data = map_s_and_b_values(data_training)
weights_signal_bs = data.loc[(data['Label'] == 1)].Weight.sum()
weights_bg_bs = data.loc[(data['Label'] == 0)].Weight.sum()
print(weights_signal_bs, weights_bg_bs)

691.9886077125524 410999.8473218613


In [30]:
rf = RandomForestClassifier()
grid_rf = {'n_estimators': [50, 200]}

best_score = 0
skf = StratifiedKFold(n_splits=3, shuffle=False)

for g in ParameterGrid(grid_rf):
    rf.set_params(**g)
    score = []
    for train_index, test_index in skf.split(X_train, y_train):
        X_train_skf, X_test_skf = X_train[train_index], X_train[test_index]
        y_train_skf, y_test_skf = y_train[train_index], y_train[test_index]
        weights_train = weights[train_index]
        weights_test = weights[test_index]
    
        # ajuste de pesos
        weight_signal_as = weights_train[y_train_skf == 1].sum()
        weight_bg_as = weights_train[y_train_skf == 0].sum()
        weights_train[y_train_skf == 1] *= (weights_signal_bs/weight_signal_as)
        weights_train[y_train_skf == 0] *= (weights_bg_bs/weight_bg_as)

        weight_signal_as = weights_test[y_test_skf == 1].sum()
        weight_bg_as = weights_test[y_test_skf == 0].sum()
        weights_test[y_test_skf == 1] *= (weights_signal_bs/weight_signal_as)
        weights_test[y_test_skf == 0] *= (weights_bg_bs/weight_bg_as)

        rf.fit(X_train_skf, y_train_skf)
        # score con AMS
        y_pred_skf = rf.predict(X_test_skf)
        s = weights_test[(y_pred_skf==1) & (y_test_skf==1)].sum()
        b = weights_test[(y_pred_skf==1) & (y_test_skf==0)].sum()
        score.append(AMS(s, b))
        print(score)
    mean_score = np.mean(score)
    if mean_score > best_score:
        best_score = mean_score
        best_grid = g

[2.797206612600598]
[2.797206612600598, 2.8000981967871534]
[2.797206612600598, 2.8000981967871534, 2.8073202951254013]
[2.856655981580789]
[2.856655981580789, 2.845690497203011]
[2.856655981580789, 2.845690497203011, 2.846780621109384]


In [31]:
print(best_score)
print(best_grid)

2.849709033297728
{'n_estimators': 200}
